In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("train.tsv", sep="\t", index_col=0)
train = train.fillna(0)

In [ ]:
train

In [ ]:
test = pd.read_csv("test.tsv", sep="\t", index_col=0)
test = test.fillna(0)

In [ ]:
train['distance'] = (train['user_lat'] - train['lat']) ** 2 + (train['user_lon'] - train['lon']) ** 2

In [ ]:
test['distance'] = (test['user_lat'] - test['lat']) ** 2 + (test['user_lon'] - test['lon']) ** 2

In [ ]:
x = train.drop(columns=['target'])

In [ ]:
y = train['target']

In [ ]:
test

In [ ]:
mins=test.groupby(['group_num'])['distance'].min()

In [ ]:
a = [test.iloc[i]['distance'] == mins[test.iloc[i]['group_num']] for i in range(len(test))]
sum(a)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cb = CatBoostClassifier(iterations=200)

In [ ]:
cat_features_idx = np.where(x.dtypes == 'object')[0].tolist()

In [ ]:
cb.fit(x, y, cat_features=cat_features_idx)

In [ ]:
output=cb.predict_proba(test)[:,1]

In [ ]:
a = test['group_num'].values

In [ ]:
a

In [ ]:
answer = []
start_index = 0
for i in range(len(a)):
    if a[i] != a[start_index]:
        end_index = i
        _min = max(output[start_index:end_index])
        for el in range(start_index, end_index):
            answer.append(1 if output[el] == _min else 0)
        start_index = i
_min = max(output[start_index:])
for el in range(start_index, len(a)):
    answer.append(1 if output[el] == _min else 0)


In [ ]:
len(answer)

In [ ]:
sum(answer)

In [ ]:
np.savetxt("target.tsv", answer, fmt='%d')